# Exploração da Base de dados do Sistema de Informações Hospitalares do SUS (SIHSUS)

Este notebook mantém o registro de algumas informações importantes do SIHSUS, obtidas durante análise exploratória, no contexto do Datathon promovido pela [SEMOB](https://simu.mdr.gov.br/datathon/).

Como a base de dados do SIHSUS é bastante extensa e está distribuída em diversos arquivos no formato CSV, construímos esta análise sobre uma pequena amostra dela. O objetivo da exploração inicial foi a construção de um script (separado) para extrair da base completa apenas as informações relevantes para o desafio proposto no Datathon.

Assim, este notebook é dividido em três momentos:
1) Carregar a amostra da base do SIHSUS em um único DataFrame;
2) Entender onde estão as informações que vinculam o registro da AIH a uma ocorrência no trânsito; e
3) Entender quais colunas carregam outras informações relevantes em nosso contexto, assim como a qualidade dos dados em cada uma delas.

A base de dados do SIHSUS é mantida pela [Plataforma de Ciência de Dados aplicada à Saúde](https://pcdas.icict.fiocruz.br/).

O dicionário de variáveis da base, de suma importância pra esta análise, é mantido em [Dicionário de variáveis do SIHSUS](https://pcdas.icict.fiocruz.br/conjunto-de-dados/sistema-de-informacoes-hospitalares-do-sus-sihsus/dicionario-de-variaveis/).

Uma outra descrição dos registros produzida pelo próprio DATASUS pode ser encontrada no [link](http://siab.datasus.gov.br/DATASUS/index.php?area=0901&&item=1&acao=13&noticia=7760)

In [1]:
from multiprocessing import Pool
import pandas as pd
import re
from zipfile import ZipFile

## Carregamento da amostra

A variável `AMOSTRA` define quais arquivos vão ser carregados no DataFrame amostral `sihsus`.

Como os arquivos seguem uma nomeclatura padrão, só é necessário especificar o Estado, ano e mês do arquivo para que ele seja carregado na amostra. Busquei uma boa representatividade entre Estados e anos.

O caminho para a base de dados completa - o arquivo zipado - é especificado no interior da função `extrair`. Um Pool de processos trabalhadores é utilizado para agilizar a extração dos arquivos.

In [2]:
AMOSTRA = [
    # UF, ANO, MÊS
    ('GO', 2008, 1),
    ('RJ', 2009, 1),
    ('PE', 2010, 1),
    ('RS', 2011, 1),
    ('PR', 2012, 1),
    ('DF', 2013, 1),
    ('MA', 2014, 1),
    ('PI', 2015, 1),
    ('BA', 2016, 1),
    ('RN', 2017, 1),
    ('MG', 2018, 1),
    ('CE', 2019, 1),
    ('MT', 2020, 1),
    ('PA', 2021, 1),
    ('AM', 2022, 1),
    ('SP', 2023, 1),
    ('RR', 2024, 1),
]

def extrair(uf, ano, mes):
    """
    Função para extrair um único arquivo da base de dados
    e retorná-lo como um DataFrame.
    """
    with ZipFile('../../etlsih/ETLSIH.zip') as z:
        with z.open(f'ETLSIH.ST_{uf}_{ano}_{mes}_t.csv') as f:
            df = pd.read_csv(f, dtype='object')
    return df

with Pool() as pool:
    sihsus = pd.concat(pool.starmap(extrair, AMOSTRA))

# Se a extração com paralelismo não funcionar, as duas linhas
# abaixo produzem o mesmo resultado de forma sequencial
# from itertools import startmap
# sihsus = pd.concat(starmap(extrair, AMOSTRA))

### Visão geral da amostra

In [3]:
sihsus.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 813840 entries, 0 to 2919
Data columns (total 235 columns):
 #    Column                       Non-Null Count   Dtype 
---   ------                       --------------   ----- 
 0    UF_ZI                        813840 non-null  object
 1    ANO_CMPT                     813840 non-null  object
 2    MES_CMPT                     813840 non-null  object
 3    ESPEC                        813840 non-null  object
 4    CGC_HOSP                     617415 non-null  object
 5    N_AIH                        813840 non-null  object
 6    IDENT                        813840 non-null  object
 7    CEP                          813840 non-null  object
 8    MUNIC_RES                    813840 non-null  object
 9    NASC                         813840 non-null  object
 10   SEXO                         813840 non-null  object
 11   UTI_MES_IN                   813840 non-null  object
 12   UTI_MES_AN                   813840 non-null  object
 13   UTI_

## Encontrando AIHs relacionadas ao trânsito

O objetivo desta seção é compreender quais colunas carregam informações que associam o registro de AIH com alguma ocorrência relacionada ao trânsito. Para isto, duas chaves de busca foram utilizadas na tabela toda. Primeiro, simplesmente procuramos em toda a base pela ocorrência da palavra "trânsito" (ignorando case). Para a segunda chave, nos baseamos nos códigos [CID-10](http://www2.datasus.gov.br/cid10/V2008/cid10.htm). De acordo com a tabela, disponibilizada pelo DATASUS, diagnósticos relacionados à acidentes com meios de transporte terrestre recebem códigos entre V01x e V89x. O último dígito do código especifica, na maior parte dos casos, se o acidente é de trânsito ou não-de-trânsito.

In [4]:
def busca_regex(elemento, regex):
    """
    Retorna True se regex puder ser encontrada no elemento.
    `regex` deve ser uma expressão compilada com re.compile().
    """
    if regex.search(elemento) is None:
        return False
    else:
        return True

### Busca pelo termo "trânsito"

In [5]:
regex = re.compile(r'trânsito', re.I)
mask = sihsus.map(busca_regex, na_action='ignore', regex=regex)
recorte = sihsus.loc[mask.any(axis='columns'), mask.any(axis='index')]

In [6]:
recorte.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2573 entries, 8639 to 171797
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   def_car_int  2573 non-null   object
dtypes: object(1)
memory usage: 40.2+ KB


In [7]:
sihsus.def_car_int.value_counts()

def_car_int
Urgência                                           642141
Eletivo                                            164485
Out tp lesões e envenen por agent quím físicos       4638
Outros tipo de acidente de trânsito                  2573
Acidente no local trabalho ou a serv da empresa         3
Name: count, dtype: int64

In [8]:
sihsus[['CAR_INT', 'def_car_int']].drop_duplicates()

,CAR_INT,def_car_int
0,02,Urgência
180,01,Eletivo
7603,03,Acidente no local trabalho ou a serv da empresa
8621,06,Out tp lesões e envenen por agent quím físicos
8639,05,Outros tipo de acidente de trânsito


In [5]:
sihsus.MORTE.value_counts()

MORTE
0    780144
1     33696
Name: count, dtype: int64

A busca pelo termo "trânsito" em toda a amostra indica que apenas a coluna `def_car_int` (definição do caráter da internação) faz uso do mesmo, e que corresponde ao código de caráter da internação "05".

### Busca por CID-10

A busca por códigos de diagnóstico ([CID-10](http://www2.datasus.gov.br/cid10/V2008/cid10.htm)) é mais complexa do que a realizada acima. Uma análise das variáveis (colunas) mantidas pela base de dados sugere o uso de múltiplos campos para anotação deste código, possívelmente em períodos diferentes. É o caso, por exemplo, das colunas `DIAGSEC[1-9]`, que passaram a ser utilizadas à partir de 2014 em detrimento da coluna `DIAG_SECUN`.

A tabela CID-10 especifica códigos na região entre V01 e V89 para acidentes com meios de transporte terrestre. Contudo, nem todos são classificados como acidentes de trânsito. O terceiro dígito do código indica, na maior parte das vezes, se é um acidente de trânsito ou não-de-trânsito. Através de uma leitura atenta da tabela foi possível construir uma expressão regular (`regex`) que abarca todos os acidentes de trânsito, incluso os não especificados se de trânsito ou não-de-trânsito.

Padrão dos códigos de diagnóstico: código de três dígitos.

In [8]:
sihsus.DIAG_SECUN.unique()

array([nan, 'N201', 'X400', ..., 'Z891', 'Z480', '0000'], dtype=object)

In [9]:
regex = re.compile(r"""^V
    ((?:[1-7][0-9][5679])  # Diversos casos de acidentes de trânsito
    |(?:0[0-8][19])        # Pedestre em colisão de trânsito
    |(?:09[23])            # Pedestre em outros acidentes de trânsito
    |(?:[1-7]94)           # Colisão com veículos não especificados
    |(?:[12][0-8]4)        # Especificidade ciclista e motociclista
    |(?:87[0-9])           # Pessoa traumatizada em acidente de trânsito
    |(?:8[3-6][0-3])       # Veículos especiais em acidente de trânsito
    |(?:89[23])            # Acidente com veículo não especificado
    )""", re.X)

mask = sihsus.map(busca_regex, na_action='ignore', regex=regex)
recorte = sihsus.loc[mask.any(axis='columns'), mask.any(axis='index')]

In [10]:
recorte.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8684 entries, 3153 to 2908
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   DIAG_PRINC  8684 non-null   object
 1   DIAG_SECUN  8678 non-null   object
 2   CID_ASSO    6940 non-null   object
 3   CID_MORTE   6937 non-null   object
 4   DIAGSEC1    6863 non-null   object
 5   DIAGSEC2    627 non-null    object
 6   DIAGSEC3    216 non-null    object
 7   DIAGSEC4    85 non-null     object
 8   DIAGSEC5    42 non-null     object
dtypes: object(9)
memory usage: 678.4+ KB


In [11]:
recorte

,DIAG_PRINC,DIAG_SECUN,CID_ASSO,CID_MORTE,DIAGSEC1,DIAGSEC2,DIAGSEC3,DIAGSEC4,DIAGSEC5
3153,S818,V039,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5827,S063,V109,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6337,S090,V299,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6436,S525,V289,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8536,S828,V299,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2877,T068,0000,0000,0000,V299,NaN,NaN,NaN,NaN
2878,T068,0000,0000,0000,V299,NaN,NaN,NaN,NaN
2881,T068,0000,0000,0000,V199,NaN,NaN,NaN,NaN
2882,S140,0000,0000,0000,V299,NaN,NaN,NaN,NaN


A busca por códigos CID-10 retornou 9 colunas que podem ser utilizadas para inclusão de um diagnóstico derivado de acidente de trânsito, incluso como diagnóstico primário. As colunas `DIAGSEC[6-9]` não foram capturadas na busca, mas pode ser uma especificidade da amostra. Por isso, é importante incluí-las no script de extração.

Por fim, é importante observarmos que os registros de AIH encontrados, de um lado, pela busca do termo "trânsito" e, de outro, pelos códigos CID-10, não se sobrepoem, já que nenhuma das buscas retorna a totalidade dos registros encontrados pela outra. Ou seja, será necessário aplicar os dois tipos de filtro para extrairmos as AIH relacionadas aos acidentes de trânsito do SIHSUS.

In [12]:
sihsus.loc[recorte.index]['def_car_int'].value_counts()

def_car_int
Urgência                                          60868
Eletivo                                           12961
Outros tipo de acidente de trânsito                1699
Out tp lesões e envenen por agent quím físicos      501
Name: count, dtype: int64

In [13]:
# Limpeza de variáveis
del regex
del mask
del recorte

## Análise de outras variáveis de interesse

O objetivo desta seção é mapear algumas variáveis interessantes para análise dos dados de internações por acidentes de trânsito. As variáveis observadas aqui serão mantidas nos dados extraídos do sihsus.

Aqui, deve ser observado que de acordo com a PCDaS:
>As colunas com nomes em MAIÚSCULO representam dados originais advindos do DATASUS e colunas com nomes iniciando em minúsculo representam dados resultantes de transformação ou enriquecimento.


Preferência será dada, onde possível, às variáveis resultantes da transformação ou enriquecimento.

### Localidade

De acordo com o [dicionário de variáveis](https://pcdas.icict.fiocruz.br/conjunto-de-dados/sistema-de-informacoes-hospitalares-do-sus-sihsus/dicionario-de-variaveis/), a base do SIHSUS mantém dois registros distintos de localidade: o município de residência do paciente internado (prefixado com `res_`) e o município da unidade hospitalar (prefixado com `int_`). Para cada um, são associados também os códigos da microregional de saúde, regional de saúde, UF, coordenadas geográficas do município e atributos indicando se é capital, na região amazônica ou na fronteira. Ambos - município de residência e município da UH - serão mantidos em nosso extrato final. 

In [14]:
mask = sihsus['res_codigo_adotado'] != sihsus['res_MUNCOD']
sihsus.loc[mask, ['res_codigo_adotado', 'res_MUNCOD', 'res_MUNNOME']].drop_duplicates()

,res_codigo_adotado,res_MUNCOD,res_MUNNOME
2727,530010,530070,Brasília
8082,530010,530150,Brasília
8119,530010,530180,Brasília
9682,530010,530140,Brasília
11328,530010,530090,Brasília
11702,530010,530170,Brasília
12420,530010,530060,Brasília
12658,530010,530040,Brasília
5,530010,530120,Brasília
11,530010,530110,Brasília


### Valores de internações

A base do SIHSUS traz diferentes colunas especificando cada componente do custo de uma internação hospitalar. Estas colunas são prefixadas com `VAL_`. Para nossa análise, é interessante observarmos os custos impostos ao sistema de saúde com internações por acidentes de trânsito. Contudo, talvez apenas o valor total seja interessante. A tabela possui uma coluna `VAL_TOT`, mas há gastos com UTI e que não estão incluídos neste valor total. Nesta seção, exploramos quais valores estão incluídos em `VAL_TOT`.

In [35]:
recorte = sihsus[[c for c in sihsus.columns if c.startswith('VAL_')]].astype(pd.Float64Dtype())

In [36]:
recorte.info()

<class 'pandas.core.frame.DataFrame'>
Index: 813840 entries, 0 to 2919
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   VAL_SH      813840 non-null  Float64
 1   VAL_SP      813840 non-null  Float64
 2   VAL_SADT    813840 non-null  Float64
 3   VAL_RN      813840 non-null  Float64
 4   VAL_ACOMP   813840 non-null  Float64
 5   VAL_ORTP    813840 non-null  Float64
 6   VAL_SANGUE  813840 non-null  Float64
 7   VAL_SADTSR  813840 non-null  Float64
 8   VAL_TRANSP  813840 non-null  Float64
 9   VAL_OBSANG  813840 non-null  Float64
 10  VAL_PED1AC  813840 non-null  Float64
 11  VAL_TOT     813840 non-null  Float64
 12  VAL_UTI     813840 non-null  Float64
 13  VAL_SH_FED  690888 non-null  Float64
 14  VAL_SP_FED  690888 non-null  Float64
 15  VAL_SH_GES  690888 non-null  Float64
 16  VAL_SP_GES  690888 non-null  Float64
 17  VAL_UCI     571021 non-null  Float64
dtypes: Float64(18)
memory usage: 131.9 MB


In [37]:
recorte.sum()

VAL_SH         789766070.4
VAL_SP        196099384.28
VAL_SADT               0.0
VAL_RN                 0.0
VAL_ACOMP              0.0
VAL_ORTP               0.0
VAL_SANGUE             0.0
VAL_SADTSR             0.0
VAL_TRANSP             0.0
VAL_OBSANG             0.0
VAL_PED1AC             0.0
VAL_TOT       988601048.95
VAL_UTI        229816754.7
VAL_SH_FED       814098.23
VAL_SP_FED       490338.36
VAL_SH_GES       803187.29
VAL_SP_GES       627970.39
VAL_UCI          6902244.2
dtype: Float64

In [42]:
recorte.sum().loc[[c for c in recorte.columns if c not in ['VAL_TOT', 'VAL_UTI', 'VAL_UCI']]].sum()

988601048.9499997

Portanto, a coluna `VAL_TOT` inclui a soma das demais, com exceção de `VAL_UTI` e `VAL_UCI`. Como as unidades de cuidado intensivo (UCI) são associadas ao tratamento de recém nascidos, não iremos incluí-la na extração final. Mas, para o cálculo dos gastos totais com internações decorrentes de acidentes de trânsito, devemos considerar as soma de `VAL_TOT` com `VAL_UTI`.